Логистическая регрессия
Обучим логистическую регрессию для предсказания того, откликнется клиент
на рекламное предложение (target = 1) или нет (target = 0).

Подключение библиотек, загрузка и обзор данных

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')
print(data.head())

# Разделяем модель selection
from sklearn.model_selection import train_test_split

X = data.drop(['TARGET'], axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

   AGE  SOCSTATUS_WORK_FL  SOCSTATUS_PENS_FL  GENDER  CHILD_TOTAL  DEPENDANTS  \
0   49                  1                  0       1            2           1   
1   32                  1                  0       1            3           3   
2   52                  1                  0       1            4           0   
3   39                  1                  0       1            1           1   
4   30                  1                  0       0            0           0   

   PERSONAL_INCOME  LOAN_NUM_TOTAL  LOAN_NUM_CLOSED  LOAN_DLQ_NUM  TARGET  
0           5000.0               1                1             2       0  
1          12000.0               1                1             1       0  
2           9000.0               2                1             0       0  
3          25000.0               1                1             3       0  
4          12000.0               2                1             2       0  


Практика
Задание

Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,
имея в виду 41 процент).


In [19]:
# Подсчёт количества объектов каждого класса
class_counts = pd.Series(ytrain).value_counts()
class_counts1 = pd.Series(ytest).value_counts()
print("Количество объектов класса 0 (отрицательный):", class_counts[0] + class_counts1[0])
print("Количество объектов класса 1 (положительный):", class_counts[1] + class_counts1[1])

Количество объектов класса 0 (отрицательный): 13411
Количество объектов класса 1 (положительный): 1812


Обучим логистическую регрессию с параметрами по умолчанию.

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Масштабируем данные
scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

lr = LogisticRegression(random_state=42, max_iter=1000)

lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)
prediction

array([0, 0, 0, ..., 0, 0, 0], shape=(4567,))

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [23]:
from sklearn.metrics import f1_score

# ваш код здесь

In [37]:
f1 = f1_score(ytest, prediction)
print("f1_score модели:", f1)

# Получаем вероятности для тестовых данных
probs_test = lr.predict_proba(Xtest_scaled)[:, 1]  # Вероятности класса 1
probs_test

f1_score модели: 0.0


array([0.16834787, 0.22328888, 0.46617787, ..., 0.25068258, 0.31759597,
       0.27000442], shape=(4567,))

Вопрос
Чему равен f1_score?

---

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [26]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]


array([[0.93059542, 0.06940458],
       [0.90718483, 0.09281517],
       [0.72665238, 0.27334762],
       [0.85108245, 0.14891755],
       [0.85995014, 0.14004986],
       [0.92562877, 0.07437123],
       [0.90219118, 0.09780882],
       [0.97635873, 0.02364127],
       [0.94283674, 0.05716326],
       [0.89995088, 0.10004912]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

Вопрос
Чему равен f1_score, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [38]:
# Применяем порог 0.1
new_predictions = (probs_test >= 0.1).astype(int)

# Считаем f1_score с новым порогом
f1_new = f1_score(ytest, new_predictions)
print("f1_score с порогом 0.1:", round(f1_new, 2))

f1_score с порогом 0.1: 0.23


(probs_test >= 0.1): Создаёт булев массив (True или False), где True — если вероятность ≥ 0.1.

У обученной модели можно посмотреть веса (как и в линейной регрессии).
Выведем на экран веса модели (model.coef_, model.intercept_).

In [39]:
lr.coef_, lr.intercept_

(array([[-2.65032335e-02, -1.68003951e-01, -8.01332228e-01,
         -8.30676970e-02,  1.45240792e-01, -5.76147961e-02,
          2.07176929e-05,  2.33261915e-01, -4.24602011e-01,
          2.30325933e-01]]),
 array([-1.21505488]))

Создайте pd.DataFrame, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).
Отсортируйте таблицу по убыванию весов.

Вопрос
Какой признак имеет наибольший положительный вес?

In [40]:
# Создаём DataFrame с весами
weights_df = pd.DataFrame({
    'Feature': Xtrain.columns,  # Названия признаков
    'Weight': lr.coef_[0]       # Веса признаков
})

# Создаём DataFrame для intercept
intercept_df = pd.DataFrame({
    'Feature': ['intercept'],
    'Weight': [lr.intercept_[0]]
})

# Объединяем с помощью pd.concat
weights_df = pd.concat([weights_df, intercept_df], ignore_index=True)

# Сортируем по убыванию весов
weights_df = weights_df.sort_values(by='Weight', ascending=False)

# Выводим таблицу
print(weights_df)

# Находим признак с наибольшим положительным весом
top_feature = weights_df[weights_df['Weight'] > 0]['Feature'].iloc[0]
print("Признак с наибольшим положительным весом:", top_feature)

              Feature    Weight
7      LOAN_NUM_TOTAL  0.233262
9        LOAN_DLQ_NUM  0.230326
4         CHILD_TOTAL  0.145241
6     PERSONAL_INCOME  0.000021
0                 AGE -0.026503
5          DEPENDANTS -0.057615
3              GENDER -0.083068
1   SOCSTATUS_WORK_FL -0.168004
8     LOAN_NUM_CLOSED -0.424602
2   SOCSTATUS_PENS_FL -0.801332
10          intercept -1.215055
Признак с наибольшим положительным весом: LOAN_NUM_TOTAL


Бонус
Задание 1
Подберите порог для перевода вероятностей в классы, дающий максимальное значение f1_score.

In [ ]:
# ваш код здесь

Задание 2
Во вложенном цикле подберите одновременно коэффициент регуляризации C у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение f1_score.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: Xtrain, Xval, Xtest.

В цикле при подборе С и порога будем обучаться по Xtrain, а предсказывать и измерять качество по Xval.

Качество итоговой модели с найденными C и порогом измерьте по Xtest.

Так не переобучимся!

In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

In [ ]:
model = LogisticRegression(C = ...)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)

classes = ... # переведите полученные вероятности в классы по найденному порогу

In [ ]:
# вычислите значение f1_score на тестовых данных